In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/traffic/retina_imgs.zip' -d ./
!unzip '/content/bgs.zip' -d ./

Archive:  /content/bgs.zip
   creating: ./content/bg/
  inflating: ./content/bg/images.jpeg  
  inflating: ./content/bg/WIN_20230317_22_39_10_Pro.jpg  
  inflating: ./content/bg/images (1).jpeg  
  inflating: ./content/bg/WIN_20230317_22_39_21_Pro.jpg  
  inflating: ./content/bg/download.jpeg  


In [ ]:
def helper(img):
  img = cv2.imread(img, 0)
  h, w = img.shape[:2]
  bg = np.random.choice(bgs)
  if np.random.randint(2): bg = cv2.GaussianBlur(img, (3,3),0)
  if np.random.randint(2):
    bg = bright_noise(bg)
    img = bright_noise(img)
  img = cv2.resize(img, (np.random.randint(int(w*0.5), int(w*1)), np.random.randint(int(h*0.5), int(h*1))))
  pt = np.random.randint(0,(1000 - max(h,w)))
  # sz = np.random.randint(800,1000)
  bg = cv2.resize(bg, (1000, 1000))
  bg = cv2.GaussianBlur(bg, (3,3), 0)
  h,w = img.shape[:2]
  img = cv2.resize(img, (np.random.randint(int(w*0.7), int(w*1.2)), np.random.randint(int(h*0.7), int(h*1.2))))
  h,w = img.shape[:2]
  pt = np.random.randint(0,(1000 - max(h,w)))
  bg[pt:pt+h, pt:pt+w] = img
  return 0, [pt, pt+w, pt, pt+h], bg

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return [x,y,w,h]

def bright_noise(retimg):
  bm = cv2.resize((np.random.rand(np.random.randint(2,20),np.random.randint(2,20))*255).astype(np.uint8),(retimg.shape[1],retimg.shape[0]),cv2.INTER_CUBIC)
  bm = cv2.normalize(bm,np.zeros(bm.shape),0,np.random.randint(0,100),cv2.NORM_MINMAX)
  retimg = retimg.astype(int)
  retimg = retimg+bm
  retimg[retimg>255]=255
  retimg = retimg.astype(np.uint8)
  retimg = cv2.normalize(retimg,np.zeros(bm.shape),0,255,cv2.NORM_MINMAX)
  return retimg

In [ ]:
bgs = [cv2.imread('/content/content/bg/'+i, 0) for i in os.listdir('/content/content/bg')]
# aspect ratio 1:1.5 to 2
import time 
import shutil
import pandas as pd
save_yolo_dir = '/content/data/'
try:
  shutil.rmtree(save_yolo_dir)
  os.mkdir(save_yolo_dir)
except:
  time.sleep(3)
  os.mkdir(save_yolo_dir)
time.sleep(3)
os.mkdir(save_yolo_dir+'images')
time.sleep(3)
os.mkdir(save_yolo_dir+'images/train/')
os.mkdir(save_yolo_dir+'images/val/')
os.mkdir(save_yolo_dir+'labels')
time.sleep(3)
os.mkdir(save_yolo_dir+'labels/train/')
os.mkdir(save_yolo_dir+'labels/val/')

# src_dir = "/content/drive/MyDrive/keys detector/keys annotations/images/train"
# dst_dir = "/content/data/images/train"
# for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
#     shutil.copy(jpgfile, dst_dir)

# src_dir = '/content/drive/MyDrive/keys detector/keys annotations_dl/keys annotations/labels/train'
# dst_dir = '/content/data/labels/train'
# for txt in glob.iglob(os.path.join(src_dir, "*.xml")):
#   shutil.copy(txt, dst_dir)
ddir = '/content/retina_imgs/'
imgss = np.array(os.listdir(ddir))
np.random.shuffle(imgss)
testing_ratio = 0.1
gen_sample = 4000
test_sample = int(gen_sample*testing_ratio)
for i in imgss[:]:
  # img = np.random.choice(bgs)
  # print(resize_pt,cropped_img.shape)
  for k in range(4):
    label, pts, final_img= helper(ddir+i)
    key_loc = [[label]+convert(final_img.shape[:2][::-1], pts)]
    key_loc = pd.DataFrame(key_loc)
    if not np.random.choice([0,1,2,3,4,5,6]):
      key_loc.to_csv('/content/data/labels/val/'+ str(i)+str(k)+'.txt', index = False, header = False, sep=' ')
      cv2.imwrite('/content/data/images/val/'+ str(i)+str(k)+'.jpg', final_img)
    else:
      key_loc.to_csv('/content/data/labels/train/'+ str(i)+str(k)+'.txt', index = False, header = False, sep=' ')
      cv2.imwrite('/content/data/images/train/'+ str(i)+str(k)+'.jpg', final_img)
    # img = x.copy()



classes_ = ['detect']
nc = len(classes_)
yamlstr = f'''train: {save_yolo_dir}images/train/  \nval: {save_yolo_dir}images/val/  \nnc: {nc}  \nnames: {classes_} '''
try:
    os.remove("/content/cc.yaml")
except:pass
file1 = open("/content/cc.yaml","a")#append mode
file1.write(yamlstr)
file1.close()

<ipython-input-11-b2c5f685909b>:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bg = np.random.choice(bgs)


In [ ]:
len(os.listdir('/content/data/labels/train/'))

147

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/78.2 GB disk)


In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 30 --epochs 40 --data /content/cc.yaml --weights yolov5m.pt --cache --hyp /content/yolov5/data/hyps/hyp.scratch-low.yaml

train: weights=yolov5m.pt, cfg=, data=/content/cc.yaml, hyp=/content/yolov5/data/hyps/hyp.scratch-low.yaml, epochs=40, batch_size=30, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-140-g1db9533 Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=35.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.3, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hs

In [ ]:
import shutil
shutil.copy('./runs/train/exp/weights/retina.pt', '/content/drive/MyDrive/malware/')

'/content/drive/MyDrive/malware/retina.pt'